In [6]:
!pip3 install torchvision

In [7]:
import cv2
import os
import zipfile
import torch
import torchvision.transforms as transforms
from PIL import Image
import numpy as np




In [8]:
extract_dir='/Users/apoorvagayatrik/PerspectAIProj/dataset'

In [9]:
import os
import glob
import cv2
import numpy as np
import torch

os.chdir('/Users/apoorvagayatrik/PerspectAIProj/ESRGAN')
import RRDBNet_arch as arch

# Define the apply_esrgan function
def apply_esrgan(img, model, device):
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round().astype(np.uint8)

    return output

In [10]:
# Load the ESRGAN model
model_path = '/Users/apoorvagayatrik/PerspectAIProj/ESRGAN/models/RRDB_ESRGAN_x4.pth'
device = torch.device('cpu')  # if you want to run on CPU, change 'cuda' -> 'cpu'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

# Example for a generic model
total_params = count_parameters(model)
print(f'Total number of parameters: {total_params}')

def get_model_size(model, filepath):
    torch.save(model.state_dict(), filepath)  # For PyTorch, or appropriate save function for other frameworks
    return os.path.getsize(filepath)

model_size = get_model_size(model, 'model.pth')
print(f'Model size: {model_size / (1024**2)} MB')






Total number of parameters: 16697987
Model size: 63.88340950012207 MB


In [2]:
def detect_faces(image):

    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    return faces

In [3]:
datasets=['train_ds','val_ds','test_ds']

train_preprocessed=[]
val_preprocessed=[]
test_preprocessed=[]


In [4]:
!pip3 install tqdm

In [9]:
import os
import cv2
import numpy as np
from tqdm import tqdm

# Assuming detect_faces and apply_esrgan functions are already defined

datasets = ['train_ds', 'val_ds', 'test_ds']
extract_dir = '/Users/apoorvagayatrik/PerspectAIProj/dataset2'  # Adjust this to your actual extract directory
output_dir = '/Users/apoorvagayatrik/PerspectAIProj/processed_images256'  # Directory to save processed images

# Initialize lists for storing preprocessed images
train_preprocessed = []
val_preprocessed = []
test_preprocessed = []

train_labels = []
val_labels = []
test_labels = []

def load_images_and_labels(dataset):
    images = []
    labels = []
    dataset_path = os.path.join(extract_dir, dataset)
    people_folders = [f for f in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, f))]

    progress_bar = tqdm(total=len(people_folders), desc=f'Processing {dataset}', unit='folder')

    for person_folder in people_folders:
        person_path = os.path.join(dataset_path, person_folder)
        image_files = [os.path.join(person_path, f) for f in os.listdir(person_path) if f.endswith('.jpg') or f.endswith('.png')]
        label = person_folder
        for image_path in image_files:
            image = cv2.imread(image_path, cv2.IMREAD_COLOR)
            faces = detect_faces(image)

            for (x, y, w, h) in faces:
                # Extract the face ROI
                face_roi = image[y:y+h, x:x+w]
                # Apply ESRGAN to the face ROI
                #enhanced_face = apply_esrgan(face_roi, model, device)

                # Convert to grayscale
                gray_face = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)

                # Resize the face to the target size
                target_size = (256, 256)
                resized_face = cv2.resize(gray_face, target_size)

                # Save processed image
                filename = os.path.basename(image_path)
                output_path = os.path.join(output_dir, dataset, person_folder, filename)
                os.makedirs(os.path.dirname(output_path), exist_ok=True)
                cv2.imwrite(output_path, resized_face)

                images.append(resized_face)
                labels.append(label)
        
        progress_bar.update(1)

    progress_bar.close()
    return images, labels

# Call the function to load and process images
train_preprocessed, train_labels = load_images_and_labels('train_ds')
val_preprocessed, val_labels = load_images_and_labels('val_ds')
test_preprocessed, test_labels = load_images_and_labels('test_ds')

print("Image loading and enhancement completed!")






Processing train_ds:   0%|                         | 0/1755 [00:00<?, ?folder/s]



Processing train_ds:   0%|                 | 1/1755 [00:00<02:57,  9.88folder/s]



Processing train_ds:   0%|                 | 2/1755 [00:00<05:58,  4.89folder/s]



Processing train_ds:   0%|                 | 3/1755 [00:00<04:58,  5.87folder/s]



Processing train_ds:   0%|                 | 4/1755 [00:00<04:46,  6.10folder/s]



Processing train_ds:   0%|                 | 6/1755 [00:00<04:24,  6.60folder/s]



Processing train_ds:   0%|                 | 8/1755 [00:01<04:52,  5.96folder/s]



Processing train_ds:   1%|                 | 9/1755 [00:01<04:33,  6.38folder/s]



Processing train_ds:   1%|                | 10/1755 [00:01<05:41,  5.10folder/s]



Processing train_ds:   1%|                | 11/1755 [00:02<06:24,  4.54folder/s]



Processing train_ds:   1%|                | 13/1755 [00:02<05:23,  5.38folder/s]



Processing train_ds:   1%|▏               | 14/1755 [00:02<05:59,  4.84f

Processing train_ds:   6%|▉              | 108/1755 [00:19<05:38,  4.86folder/s]



Processing train_ds:   6%|▉              | 109/1755 [00:20<05:26,  5.04folder/s]



Processing train_ds:   6%|▉              | 110/1755 [00:20<06:42,  4.09folder/s]



Processing train_ds:   6%|▉              | 111/1755 [00:20<06:42,  4.09folder/s]



Processing train_ds:   6%|▉              | 112/1755 [00:20<06:20,  4.32folder/s]



Processing train_ds:   6%|▉              | 113/1755 [00:21<06:27,  4.24folder/s]



Processing train_ds:   7%|▉              | 115/1755 [00:21<06:05,  4.49folder/s]



Processing train_ds:   7%|▉              | 116/1755 [00:21<07:05,  3.85folder/s]



Processing train_ds:   7%|█              | 118/1755 [00:22<06:15,  4.36folder/s]



Processing train_ds:   7%|█              | 119/1755 [00:22<06:16,  4.34folder/s]



Processing train_ds:   7%|█              | 120/1755 [00:22<05:34,  4.89folder/s]



Processing train_ds:   7%|█              | 121/1755 [00:22<05:25,  5.02folde

Processing train_ds:  13%|█▉             | 231/1755 [00:39<03:24,  7.44folder/s]



Processing train_ds:  13%|█▉             | 232/1755 [00:39<03:29,  7.28folder/s]



Processing train_ds:  13%|██             | 235/1755 [00:39<02:08, 11.79folder/s]



Processing train_ds:  14%|██             | 238/1755 [00:40<02:34,  9.82folder/s]



Processing train_ds:  14%|██             | 240/1755 [00:40<02:18, 10.95folder/s]



Processing train_ds:  14%|██             | 242/1755 [00:40<02:17, 10.97folder/s]



Processing train_ds:  14%|██             | 244/1755 [00:40<02:51,  8.79folder/s]



Processing train_ds:  14%|██             | 246/1755 [00:41<03:36,  6.98folder/s]



Processing train_ds:  14%|██             | 247/1755 [00:41<03:38,  6.91folder/s]



Processing train_ds:  14%|██▏            | 249/1755 [00:41<03:20,  7.50folder/s]



Processing train_ds:  14%|██▏            | 250/1755 [00:41<04:09,  6.03folder/s]



Processing train_ds:  14%|██▏            | 251/1755 [00:42<04:43,  5.30folde

Processing train_ds:  20%|███            | 352/1755 [00:58<03:52,  6.03folder/s]



Processing train_ds:  20%|███            | 353/1755 [00:58<03:33,  6.55folder/s]



Processing train_ds:  20%|███            | 355/1755 [00:58<02:33,  9.10folder/s]



Processing train_ds:  20%|███            | 357/1755 [00:59<03:38,  6.39folder/s]



Processing train_ds:  20%|███            | 358/1755 [00:59<03:33,  6.55folder/s]



Processing train_ds:  20%|███            | 359/1755 [00:59<03:51,  6.03folder/s]



Processing train_ds:  21%|███            | 361/1755 [00:59<03:41,  6.28folder/s]



Processing train_ds:  21%|███            | 362/1755 [00:59<03:51,  6.03folder/s]



Processing train_ds:  21%|███            | 363/1755 [01:00<03:39,  6.34folder/s]



Processing train_ds:  21%|███            | 364/1755 [01:00<04:28,  5.18folder/s]



Processing train_ds:  21%|███            | 365/1755 [01:00<05:28,  4.23folder/s]



Processing train_ds:  21%|███▏           | 366/1755 [01:00<04:50,  4.78folde

Processing train_ds:  26%|███▉           | 463/1755 [01:17<03:25,  6.28folder/s]



Processing train_ds:  26%|███▉           | 464/1755 [01:18<04:31,  4.76folder/s]



Processing train_ds:  26%|███▉           | 465/1755 [01:18<04:28,  4.80folder/s]



Processing train_ds:  27%|███▉           | 466/1755 [01:18<03:50,  5.59folder/s]



Processing train_ds:  27%|███▉           | 467/1755 [01:18<03:58,  5.41folder/s]



Processing train_ds:  27%|████           | 469/1755 [01:19<03:20,  6.41folder/s]



Processing train_ds:  27%|████           | 470/1755 [01:19<03:14,  6.60folder/s]



Processing train_ds:  27%|████           | 471/1755 [01:19<04:18,  4.97folder/s]



Processing train_ds:  27%|████           | 472/1755 [01:19<04:47,  4.46folder/s]



Processing train_ds:  27%|████           | 473/1755 [01:20<05:25,  3.94folder/s]



Processing train_ds:  27%|████           | 475/1755 [01:20<03:30,  6.09folder/s]



Processing train_ds:  27%|████           | 476/1755 [01:20<04:29,  4.74folde

...

Processing train_ds:  95%|█████████████▎| 1671/1755 [04:33<00:08,  9.36folder/s]



Processing train_ds:  95%|█████████████▎| 1673/1755 [04:34<00:13,  6.02folder/s]



Processing train_ds:  95%|█████████████▎| 1674/1755 [04:34<00:13,  6.21folder/s]



Processing train_ds:  95%|█████████████▎| 1676/1755 [04:34<00:10,  7.73folder/s]



Processing train_ds:  96%|█████████████▍| 1677/1755 [04:34<00:13,  5.59folder/s]



Processing train_ds:  96%|█████████████▍| 1678/1755 [04:34<00:13,  5.67folder/s]



Processing train_ds:  96%|█████████████▍| 1679/1755 [04:35<00:16,  4.58folder/s]



Processing train_ds:  96%|█████████████▍| 1681/1755 [04:35<00:15,  4.93folder/s]



Processing train_ds:  96%|█████████████▍| 1682/1755 [04:35<00:15,  4.81folder/s]



Processing train_ds:  96%|█████████████▍| 1683/1755 [04:35<00:13,  5.50folder/s]



Processing train_ds:  96%|█████████████▍| 1684/1755 [04:36<00:11,  6.14folder/s]



Processing train_ds:  96%|█████████████▍| 1685/1755 [04:36<00:11,  5.94folde

Processing val_ds:  33%|██████▋             | 17/51 [00:28<00:59,  1.74s/folder]



Processing val_ds:  35%|███████             | 18/51 [00:31<01:01,  1.86s/folder]



Processing val_ds:  37%|███████▍            | 19/51 [00:34<01:11,  2.23s/folder]



Processing val_ds:  39%|███████▊            | 20/51 [00:34<00:55,  1.80s/folder]



Processing val_ds:  41%|████████▏           | 21/51 [00:38<01:05,  2.17s/folder]



Processing val_ds:  43%|████████▋           | 22/51 [00:39<00:53,  1.86s/folder]



Processing val_ds:  45%|█████████           | 23/51 [00:40<00:43,  1.56s/folder]



Processing val_ds:  47%|█████████▍          | 24/51 [00:40<00:35,  1.33s/folder]



Processing val_ds:  49%|█████████▊          | 25/51 [00:43<00:46,  1.80s/folder]



Processing val_ds:  51%|██████████▏         | 26/51 [00:46<00:52,  2.11s/folder]



Processing val_ds:  53%|██████████▌         | 27/51 [00:47<00:42,  1.77s/folder]



Processing val_ds:  55%|██████████▉         | 28/51 [00:48<00:36,  1.59s/fol

Processing test_ds:  34%|██████            | 50/148 [00:33<01:11,  1.38folder/s]



Processing test_ds:  34%|██████▏           | 51/148 [00:34<01:05,  1.48folder/s]



Processing test_ds:  35%|██████▎           | 52/148 [00:35<01:07,  1.42folder/s]



Processing test_ds:  36%|██████▍           | 53/148 [00:35<01:05,  1.46folder/s]



Processing test_ds:  36%|██████▌           | 54/148 [00:36<01:01,  1.52folder/s]



Processing test_ds:  37%|██████▋           | 55/148 [00:36<01:00,  1.54folder/s]



Processing test_ds:  38%|██████▊           | 56/148 [00:37<01:01,  1.49folder/s]



Processing test_ds:  39%|██████▉           | 57/148 [00:38<01:01,  1.48folder/s]



Processing test_ds:  39%|███████           | 58/148 [00:38<00:58,  1.53folder/s]



Processing test_ds:  40%|███████▏          | 59/148 [00:39<00:59,  1.51folder/s]



Processing test_ds:  41%|███████▎          | 60/148 [00:40<00:54,  1.61folder/s]



Processing test_ds:  41%|███████▍          | 61/148 [00:40<00:57,  1.52folde

Processing test_ds:  91%|███████████████▌ | 135/148 [01:29<00:08,  1.46folder/s]



Processing test_ds:  92%|███████████████▌ | 136/148 [01:29<00:08,  1.49folder/s]



Processing test_ds:  93%|███████████████▋ | 137/148 [01:30<00:07,  1.56folder/s]



Processing test_ds:  93%|███████████████▊ | 138/148 [01:31<00:06,  1.50folder/s]



Processing test_ds:  94%|███████████████▉ | 139/148 [01:31<00:05,  1.54folder/s]



Processing test_ds:  95%|████████████████ | 140/148 [01:32<00:05,  1.40folder/s]



Processing test_ds:  95%|████████████████▏| 141/148 [01:33<00:04,  1.49folder/s]



Processing test_ds:  96%|████████████████▎| 142/148 [01:33<00:04,  1.47folder/s]



Processing test_ds:  97%|████████████████▍| 143/148 [01:34<00:03,  1.45folder/s]



Processing test_ds:  97%|████████████████▌| 144/148 [01:35<00:02,  1.48folder/s]



Processing test_ds:  98%|████████████████▋| 145/148 [01:35<00:02,  1.48folder/s]



Processing test_ds:  99%|████████████████▊| 146/148 [01:36<00:01,  1.58folde

Image loading and enhancement completed!


In [2]:
import torch

# Check if CUDA (GPU acceleration) is available
cuda_available = torch.cuda.is_available()

if cuda_available:
    # Get the CUDA device count
    cuda_device_count = torch.cuda.device_count()
    print('CUDA is available with {cuda_device_count} CUDA device(s)!')
else:
    print('CUDA is not available. Running on CPU.')


CUDA is not available. Running on CPU.
